In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

llm = ChatOpenAI(model='gpt-4o-mini')

llm.invoke([HumanMessage("잘지냈어?")])

AIMessage(content='네, 잘 지냈어요! 당신은 어떻게 지내고 계신가요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 12, 'total_tokens': 30, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_373a14eb6f', 'id': 'chatcmpl-DAXS72O0gW2CWlGSFigW4QMq0lwZV', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c6feb-b1e8-7503-89af-83fc87c9371d-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 12, 'output_tokens': 18, 'total_tokens': 30, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [2]:
from langchain_core.tools import tool
from datetime import datetime
import pytz

@tool # @tool 데코레이터를 사용하여 함수를 도구로 등록 
def get_current_time(timezone: str, location: str) -> str:
    """ 현재 시각을 반환하는 함수
    
    Args:
        timezone (str): 타임존(예: Asia/Seoul). 실제 존재해야 함 
        location (str): 지역명. 타임존은 모든 지명에 대응하지 않으므로 이후 llm 답변 생성에 사용됨
    """

    tz = pytz.timezone(timezone)

    now = datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')

    location_and_local_time = f'{timezone} ({location}) 현재 시각 {now}' # 타임존, 지역명, 현재 시각을 문자열로 반환

    print(location_and_local_time)

    return location_and_local_time

In [3]:
# 도구를 tools 리스트에 추가하고, tool_dict에도 추가 
tools = [get_current_time,]
tool_dict = {"get_current_time": get_current_time,}

# 도구를 모델에 바인딩 : 모델에 도구를 바인딩하면, 도구를 사용하여 답변을 생성할 수 있음 
llm_with_tools = llm.bind_tools(tools)

In [4]:
from langchain_core.messages import SystemMessage

messages = [
    SystemMessage("너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다."),
    HumanMessage("부산은 지금 몇시야?")
]

response = llm_with_tools.invoke(messages)
messages.append(response)

print(messages)

[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='부산은 지금 몇시야?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 129, 'total_tokens': 152, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_324beb61c9', 'id': 'chatcmpl-DAXSOP8vn6ilQlBmgVH2dyXzIdbZI', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c6feb-f355-7f33-b79c-559501c7e8c0-0', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': '부산'}, 'id': 'call_WecSU0OBpSv5p19Q9iCQMKLL', 'type': 'tool_call'}],

In [5]:
for tool_call in response.tool_calls:
    selected_tool = tool_dict[tool_call['name']]
    print(tool_call['args'])
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

{'timezone': 'Asia/Seoul', 'location': '부산'}
Asia/Seoul (부산) 현재 시각 2026-02-18 17:44:25


[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='부산은 지금 몇시야?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 129, 'total_tokens': 152, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_324beb61c9', 'id': 'chatcmpl-DAXSOP8vn6ilQlBmgVH2dyXzIdbZI', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c6feb-f355-7f33-b79c-559501c7e8c0-0', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': '부산'}, 'id': 'call_WecSU0OBpSv5p19Q9iCQMKLL', 'type': 'tool_call'}

In [6]:
llm_with_tools.invoke(messages)

AIMessage(content='부산의 현재 시각은 2026년 2월 18일 17시 44분입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 185, 'total_tokens': 211, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_324beb61c9', 'id': 'chatcmpl-DAXSYd6aGBJa4IC3Q2hW13vksILhP', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c6fec-179e-7fc3-9254-15521b66122d-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 185, 'output_tokens': 26, 'total_tokens': 211, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [7]:
from pydantic import BaseModel, Field

class StockHistoryInput(BaseModel):
    ticker: str = Field(..., title="주식 코드", description="주식 코드 (예: AAPL)")
    period: str = Field(..., title="기간", description="주식 데이터 조회 기간 (예: id, 1mo, 1y)")

In [8]:
import yfinance as yf

@tool
def get_yf_stock_history(stock_history_input: StockHistoryInput) -> str:
    """주식 종목의 가격 데이터를 조회하는 함수"""
    stock = yf.Ticker(stock_history_input.ticker)
    history = stock.history(period=stock_history_input.period)
    history_md = history.to_markdown()

    return history_md

tools = [get_current_time, get_yf_stock_history]
tool_dict = {
    "get_current_time": get_current_time,
    "get_yf_stock_history": get_yf_stock_history
}

llm_with_tools = llm.bind_tools(tools)

In [9]:
messages.append(HumanMessage("테슬라는 한 달 전에 비해 주가각 올랐나 내렸나?"))

response = llm_with_tools.invoke(messages)

print(response)

messages.append(response)

content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 274, 'total_tokens': 301, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_373a14eb6f', 'id': 'chatcmpl-DAXSknmo2HIqIoRzhHkXYfwL6POkR', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--019c6fec-4978-78b2-a37a-a9eb13f2f846-0' tool_calls=[{'name': 'get_yf_stock_history', 'args': {'stock_history_input': {'ticker': 'TSLA', 'period': '1mo'}}, 'id': 'call_PL62zVG5BZMiWHfD4agUH4f3', 'type': 'tool_call'}] invalid_tool_calls=[] usage_metadata={'input_tokens': 274, 'output_tokens': 27, 'total_tokens': 301, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audi

In [10]:
for tool_call in response.tool_calls:
    selected_tool = tool_dict[tool_call['name']]
    print(tool_call['args'])
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)
    print(tool_msg)

{'stock_history_input': {'ticker': 'TSLA', 'period': '1mo'}}
content='| Date                      |   Open |   High |    Low |   Close |      Volume |   Dividends |   Stock Splits |\n|:--------------------------|-------:|-------:|-------:|--------:|------------:|------------:|---------------:|\n| 2026-01-20 00:00:00-05:00 | 429.36 | 430.73 | 417.44 |  419.25 | 6.31873e+07 |           0 |              0 |\n| 2026-01-21 00:00:00-05:00 | 421.66 | 438.2  | 419.62 |  431.44 | 6.8124e+07  |           0 |              0 |\n| 2026-01-22 00:00:00-05:00 | 435.16 | 449.5  | 432.63 |  449.36 | 7.15467e+07 |           0 |              0 |\n| 2026-01-23 00:00:00-05:00 | 447.43 | 452.43 | 444.04 |  449.06 | 5.67714e+07 |           0 |              0 |\n| 2026-01-26 00:00:00-05:00 | 445    | 445.04 | 434.28 |  435.2  | 4.93974e+07 |           0 |              0 |\n| 2026-01-27 00:00:00-05:00 | 437.41 | 437.52 | 430.69 |  430.9  | 3.77331e+07 |           0 |              0 |\n| 2026-01-28 00:00:00-05:0

In [11]:
llm_with_tools.invoke(messages)

AIMessage(content='한 달 전인 2026년 1월 20일 테슬라의 주가는 $419.25에 마감하였고, 현재 주가는 $410.63입니다. 따라서 테슬라의 주가는 한 달 사이에 약간 하락했습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 1517, 'total_tokens': 1575, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_373a14eb6f', 'id': 'chatcmpl-DAXSvyYEPdswAyd9V5Prq8ykRSJYa', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c6fec-7430-7413-81f4-8b60274198c0-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 1517, 'output_tokens': 58, 'total_tokens': 1575, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})